In [3]:
import tensorflow as tf

def weighted_binary_crossentropy(weight_0, weight_1):
    # Convert weights to float32 to ensure compatibility with TensorFlow operations
    weight_0 = tf.constant(weight_0, dtype=tf.float32)
    weight_1 = tf.constant(weight_1, dtype=tf.float32)
    
    def loss(y_true, y_pred):
        # Ensure y_true is also in float32 for compatibility
        y_true = tf.cast(y_true, tf.float32)
        # Clip predictions to prevent log(0) errors
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        # Calculate binary cross-entropy loss for each class with weights
        loss_0 = -weight_0 * y_true * tf.math.log(y_pred)
        loss_1 = -weight_1 * (1 - y_true) * tf.math.log(1 - y_pred)
        # Combine the losses
        return tf.reduce_mean(loss_0 + loss_1)
    
    return loss


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Create a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, weights=[0.9, 0.1], random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model with the custom loss function
weight_0 = 0.1  # Weight for majority class
weight_1 = 0.9  # Weight for minority class
custom_loss = weighted_binary_crossentropy(weight_0, weight_1)
model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
25/25 [==============================] - 1s 6ms/step - loss: 0.4595 - accuracy: 0.7088 - val_loss: 0.3356 - val_accuracy: 0.8500
Epoch 2/100
25/25 [==============================] - 0s 2ms/step - loss: 0.2415 - accuracy: 0.8938 - val_loss: 0.1746 - val_accuracy: 0.9000
Epoch 3/100
25/25 [==============================] - 0s 2ms/step - loss: 0.1280 - accuracy: 0.8963 - val_loss: 0.1001 - val_accuracy: 0.9000
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0791 - accuracy: 0.8963 - val_loss: 0.0704 - val_accuracy: 0.9000
Epoch 5/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0592 - accuracy: 0.8963 - val_loss: 0.0581 - val_accuracy: 0.9000
Epoch 6/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0506 - accuracy: 0.8963 - val_loss: 0.0523 - val_accuracy: 0.9000
Epoch 7/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0464 - accuracy: 0.8963 - val_loss: 0.0494 - val_accuracy: 0.9000
Epoch 8/100
2

In [10]:
# Compile the model with standard binary cross-entropy loss
model_standard = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
model_standard.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_standard.fit(X_train, y_train, epochs=75, validation_data=(X_test, y_test))


Epoch 1/75
25/25 [==============================] - 1s 6ms/step - loss: 0.5317 - accuracy: 0.7800 - val_loss: 0.4252 - val_accuracy: 0.9000
Epoch 2/75
25/25 [==============================] - 0s 2ms/step - loss: 0.3560 - accuracy: 0.9013 - val_loss: 0.3287 - val_accuracy: 0.9050
Epoch 3/75
25/25 [==============================] - 0s 2ms/step - loss: 0.2880 - accuracy: 0.8988 - val_loss: 0.2885 - val_accuracy: 0.9050
Epoch 4/75
25/25 [==============================] - 0s 2ms/step - loss: 0.2502 - accuracy: 0.9038 - val_loss: 0.2689 - val_accuracy: 0.9150
Epoch 5/75
25/25 [==============================] - 0s 2ms/step - loss: 0.2226 - accuracy: 0.9137 - val_loss: 0.2578 - val_accuracy: 0.9150
Epoch 6/75
25/25 [==============================] - 0s 2ms/step - loss: 0.2026 - accuracy: 0.9212 - val_loss: 0.2499 - val_accuracy: 0.9100
Epoch 7/75
25/25 [==============================] - 0s 2ms/step - loss: 0.1867 - accuracy: 0.9287 - val_loss: 0.2458 - val_accuracy: 0.9150
Epoch 8/75
25/25 [==

In [11]:
from sklearn.metrics import classification_report

# Predictions with the custom loss model
y_pred_custom = (model.predict(X_test) > 0.5).astype("int32")
print("Custom Loss Model Performance:")
print(classification_report(y_test, y_pred_custom))

# Predictions with the standard loss model
y_pred_standard = (model_standard.predict(X_test) > 0.5).astype("int32")
print("Standard Loss Model Performance:")
print(classification_report(y_test, y_pred_standard))


7/7 [==============================] - 0s 834us/step
Custom Loss Model Performance:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       180
           1       0.62      0.25      0.36        20

    accuracy                           0.91       200
   macro avg       0.77      0.62      0.65       200
weighted avg       0.89      0.91      0.89       200

7/7 [==============================] - 0s 667us/step
Standard Loss Model Performance:
              precision    recall  f1-score   support

           0       0.93      0.96      0.95       180
           1       0.50      0.40      0.44        20

    accuracy                           0.90       200
   macro avg       0.72      0.68      0.69       200
weighted avg       0.89      0.90      0.89       200

